## Task 1: Data Import, Cleaning, and Spatial Integration

In [ ]:

import geopandas as gpd
# Load SA2 shapefile (make sure to download and unzip it from ABS)
gdf_sa2 = gpd.read_file("SA2_2021_AUST_GDA2020/SA2_2021_AUST_GDA2020.shp")
gdf_sa2 = gdf_sa2[gdf_sa2["GCC_NAME21"] == "Greater Sydney"].to_crs(epsg=4326)
gdf_sa2.to_postgis("sa2_geom", con=engine, if_exists="replace", index=False)


In [ ]:

from shapely.geometry import Point

df_stops = pd.read_csv("stops.txt")
df_stops.dropna(subset=["stop_lat", "stop_lon"], inplace=True)
geometry = [Point(xy) for xy in zip(df_stops.stop_lon, df_stops.stop_lat)]
gdf_stops = gpd.GeoDataFrame(df_stops, geometry=geometry, crs="EPSG:4326")
gdf_stops.to_postgis("gtfs_stops", con=engine, if_exists="replace", index=False)


In [ ]:

# Example query to check SRID in PostgreSQL
with engine.connect() as conn:
    for table in ["sa2_geom", "gtfs_stops", "school_catchments"]:
        result = conn.execute(f"SELECT ST_SRID(geom) FROM {table} LIMIT 1;")
        srid = result.fetchone()[0]
        print(f"SRID for {table}: {srid}")


In [ ]:

# Example: join POIs to SA2
query = '''
SELECT s.sa2_name21, COUNT(p.id) AS poi_count
FROM sa2_geom s
JOIN pois p
ON ST_Within(p.geom, s.geom)
GROUP BY s.sa2_name21;
'''
df_poi_counts = pd.read_sql(query, con=engine)
df_poi_counts.head()


In [ ]:

with engine.connect() as conn:
    conn.execute("CREATE INDEX IF NOT EXISTS idx_sa2_geom_geom ON sa2_geom USING GIST(geom);")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_gtfs_stops_geom ON gtfs_stops USING GIST(geom);")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_school_catchments_geom ON school_catchments USING GIST(geom);")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_pois_geom ON pois USING GIST(geom);")
print("Spatial indexes created.")


## Task 2: NSW POI API Extraction

In [ ]:

import geopandas as gpd
import pandas as pd
import requests
import time
from shapely.geometry import Point

# Load SA2 geometries
sa2_gdf = gpd.read_file("SA2_2021_AUST_GDA2020/SA2_2021_AUST_GDA2020.shp").to_crs(epsg=4283)

# Select target SA4 region
target_sa4 = "102"  # Example: Central Coast
sa2_in_sa4 = sa2_gdf[sa2_gdf["SA4_CODE21"] == target_sa4]
print(f"Processing {len(sa2_in_sa4)} SA2 regions in SA4 code {target_sa4}")


In [ ]:

# Define POI fetch function
def fetch_poi(minx, miny, maxx, maxy):
    url = "https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query"
    params = {
        "where": "1=1",
        "geometry": f"{minx},{miny},{maxx},{maxy}",
        "geometryType": "esriGeometryEnvelope",
        "spatialRel": "esriSpatialRelIntersects",
        "inSR": 4283,
        "outFields": "*",
        "outSR": 4283,
        "f": "json"
    }
    response = requests.get(url, params=params, timeout=30)
    response.raise_for_status()
    return response.json()


In [ ]:

# Loop through each SA2 and collect POIs
poi_results = []
for _, row in sa2_in_sa4.iterrows():
    minx, miny, maxx, maxy = row.geometry.bounds
    print(f"Fetching POIs for SA2: {row['SA2_NAME21']}...")
    data = fetch_poi(minx, miny, maxx, maxy)
    features = data.get("features", [])
    for feature in features:
        attrs = feature.get("attributes", {})
        geom = feature.get("geometry", {})
        attrs["longitude"] = geom.get("x")
        attrs["latitude"] = geom.get("y")
        poi_results.append(attrs)
    time.sleep(1)


In [ ]:

# Convert to GeoDataFrame and save to PostgreSQL
df_pois = pd.DataFrame(poi_results)
df_pois.dropna(subset=["latitude", "longitude"], inplace=True)
geometry = [Point(xy) for xy in zip(df_pois.longitude, df_pois.latitude)]
gdf_pois = gpd.GeoDataFrame(df_pois, geometry=geometry, crs="EPSG:4283")
gdf_pois.to_postgis("pois", con=engine, if_exists="replace", index=False)
print("POIs saved to PostGIS.")


## Task 3: Score Calculation

In [ ]:

# Compute POIs per SA2
query = '''
SELECT s.sa2_code21, COUNT(p.*) AS poi_count
FROM sa2_geom s
JOIN pois p ON ST_Within(p.geom, s.geom)
GROUP BY s.sa2_code21
'''
df_poi_count = pd.read_sql(query, con=engine)
df_poi_count.to_sql("poi_stats", con=engine, if_exists="replace", index=False)


In [ ]:

# Create z-score tables (assuming similar tables exist for business, stops, schools)
with engine.connect() as conn:
    conn.execute("""
        DROP VIEW IF EXISTS sa2_scores;
        CREATE VIEW sa2_scores AS
        WITH z_business AS (
            SELECT sa2_code, (business_per_1000 - AVG(business_per_1000) OVER()) / STDDEV(business_per_1000) OVER() AS z_biz
            FROM business_stats
        ),
        z_stops AS (
            SELECT sa2_code, (stop_count - AVG(stop_count) OVER()) / STDDEV(stop_count) OVER() AS z_stop
            FROM stop_stats
        ),
        z_schools AS (
            SELECT sa2_code, (school_metric - AVG(school_metric) OVER()) / STDDEV(school_metric) OVER() AS z_school
            FROM school_stats
        ),
        z_poi AS (
            SELECT sa2_code21 AS sa2_code, (poi_count - AVG(poi_count) OVER()) / STDDEV(poi_count) OVER() AS z_poi
            FROM poi_stats
        )
        SELECT
            z_business.sa2_code,
            z_biz, z_stop, z_school, z_poi,
            (z_biz + z_stop + z_school + z_poi) AS z_sum
        FROM z_business
        JOIN z_stops USING (sa2_code)
        JOIN z_schools USING (sa2_code)
        JOIN z_poi USING (sa2_code)
    """)


In [ ]:

# Fetch z_sum from sa2_scores and apply sigmoid
df_score = pd.read_sql("SELECT * FROM sa2_scores", con=engine)
df_score["score"] = expit(df_score["z_sum"])
df_score.to_sql("final_scores", con=engine, if_exists="replace", index=False)
df_score.sort_values("score", ascending=False).head()
